# [RTS-GMLC](https://github.com/gridmod/rts-gmlc) & [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

This notebook demonstrates how to run production cost modeling and other simulations on the RTS-GMLC data using PowerSimulations.jl

*This notebook was developed on 9/12/2019 using [a development branch of PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl/commit/c8e8bb5b02b149351c4b5036e8349b971aa2e3c0) - stability is unlikely*


In [1]:
] activate env

Activating environment at `~/Documents/repos/Examples/env/Project.toml`


In [2]:
] st

    Status `~/Documents/repos/Examples/env/Project.toml`
  [5ae59095] Colors v0.9.6
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.19.4
  [e2685f51] ECOS v0.10.0
  [60bf3e95] GLPK v0.11.3
  [b6b21f68] Ipopt v0.6.0
  [4076af6c] JuMP v0.20.0
  [51fcb6bd] NamedColors v0.2.0
  [f0f68f2c] PlotlyJS v0.12.5
  [91a5bcdd] Plots v0.26.3
  [e690365d] PowerSimulations v0.2.0 #jd/sim_update (https://github.com/NREL/PowerSimulations.jl.git)
  [bcd98974] PowerSystems v0.4.0 #dev (https://github.com/NREL/PowerSystems.jl.git)
  [9e3dc215] TimeSeries v0.16.0
  [0f1e0344] WebIO v0.8.11


In [4]:
using PowerSystems
using PowerSimulations
const PSY = PowerSystems
const PSI = PowerSimulations

┌ Info: Recompiling stale cache file /Users/cbarrows/.julia/compiled/v1.2/PowerSimulations/ixScC.ji for PowerSimulations [e690365d-45e2-57bb-ac84-44ba829e73c4]
└ @ Base loading.jl:1240


PowerSimulations

In [5]:
using JuMP
using Dates

In [6]:
# not working
using Xpress
Xpress_optimizer = JuMP.with_optimizer(Xpress.Optimizer,OUTPUTLOG = 1,MIPTOL=1e-6,FEASTOL=1e-6)


ArgumentError: ArgumentError: Package Xpress not found in current path:
- Run `import Pkg; Pkg.add("Xpress")` to install the Xpress package.


In [7]:
using GLPK
GLPK_optimizer = JuMP.with_optimizer(GLPK.Optimizer)

OptimizerFactory(GLPK.Optimizer, (), Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}())

## Get the RTS data

In [8]:
RTSDIR = "../RTS-GMLC/RTS_Data/SourceData/."

"../RTS-GMLC/RTS_Data/SourceData/."

In [9]:
rts_data = PSY.PowerSystemRaw(RTSDIR, 
    100.0,joinpath(RTSDIR,"../FormattedData/SIIP/user_descriptors.yaml"))

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:151
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:156
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:151
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:156
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:151
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:156
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:151
┌ Info: Successfully parsed 

PowerSystems.PowerSystemRaw(100.0, 120×14 DataFrames.DataFrame. Omitted printing of 7 columns
│ Row │ UID    │ From Bus │ To Bus │ R       │ X       │ B       │ Cont Rating │
│     │ String │ Int64    │ Int64  │ Float64 │ Float64 │ Float64 │ Int64       │
├─────┼────────┼──────────┼────────┼─────────┼─────────┼─────────┼─────────────┤
│ 1   │ A1     │ 101      │ 102    │ 0.003   │ 0.014   │ 0.461   │ 175         │
│ 2   │ A2     │ 101      │ 103    │ 0.055   │ 0.211   │ 0.057   │ 175         │
│ 3   │ A3     │ 101      │ 105    │ 0.022   │ 0.085   │ 0.023   │ 175         │
│ 4   │ A4     │ 102      │ 104    │ 0.033   │ 0.127   │ 0.034   │ 175         │
│ 5   │ A5     │ 102      │ 106    │ 0.05    │ 0.192   │ 0.052   │ 175         │
│ 6   │ A6     │ 103      │ 109    │ 0.031   │ 0.119   │ 0.032   │ 175         │
│ 7   │ A7     │ 103      │ 124    │ 0.002   │ 0.084   │ 0.0     │ 400         │
│ 8   │ A8     │ 104      │ 109    │ 0.027   │ 0.104   │ 0.028   │ 175         │
│ 9   │ A9     

In [10]:
sys_DA = System(rts_data; forecast_resolution = Dates.Hour(1))

┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystem

┌ Warning: Did not find component for forecast category=Generator name=212_CSP_HEAD_STORAGE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/forecast_parser.jl:231
┌ Warning: Did not find component for forecast category=Generator name=212_CSP_HEAD_STORAGE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/forecast_parser.jl:231


System(Dict{DataType,Dict{String,#s278} where #s278<:Component}(LoadZones => Dict("1" => LoadZones(name="1"),"2" => LoadZones(name="2"),"3" => LoadZones(name="3")),Line => Dict("C31-2" => Line(name="C31-2"),"B8" => Line(name="B8"),"B26" => Line(name="B26"),"A32-2" => Line(name="A32-2"),"CA-1" => Line(name="CA-1"),"A5" => Line(name="A5"),"B34" => Line(name="B34"),"B19" => Line(name="B19"),"B31-2" => Line(name="B31-2"),"C30" => Line(name="C30")…),HVDCLine => Dict("DC1" => HVDCLine(name="DC1")),HydroDispatch => Dict("215_HYDRO_3" => HydroDispatch(name="215_HYDRO_3"),"215_HYDRO_2" => HydroDispatch(name="215_HYDRO_2"),"222_HYDRO_5" => HydroDispatch(name="222_HYDRO_5"),"122_HYDRO_2" => HydroDispatch(name="122_HYDRO_2"),"322_HYDRO_2" => HydroDispatch(name="322_HYDRO_2"),"222_HYDRO_2" => HydroDispatch(name="222_HYDRO_2"),"122_HYDRO_6" => HydroDispatch(name="122_HYDRO_6"),"322_HYDRO_3" => HydroDispatch(name="322_HYDRO_3"),"322_HYDRO_4" => HydroDispatch(name="322_HYDRO_4"),"222_HYDRO_6" => Hydro

In [11]:
sys_RT = System(rts_data; forecast_resolution = Dates.Minute(5))

┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/cdm_parser.jl:845
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystem

┌ Warning: Did not find component for forecast category=Generator name=212_CSP_HEAD_STORAGE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/forecast_parser.jl:231
┌ Warning: Did not find component for forecast category=Generator name=212_CSP_HEAD_STORAGE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/parsers/forecast_parser.jl:231


System(Dict{DataType,Dict{String,#s278} where #s278<:Component}(LoadZones => Dict("1" => LoadZones(name="1"),"2" => LoadZones(name="2"),"3" => LoadZones(name="3")),Line => Dict("C31-2" => Line(name="C31-2"),"B8" => Line(name="B8"),"B26" => Line(name="B26"),"A32-2" => Line(name="A32-2"),"CA-1" => Line(name="CA-1"),"A5" => Line(name="A5"),"B34" => Line(name="B34"),"B19" => Line(name="B19"),"B31-2" => Line(name="B31-2"),"C30" => Line(name="C30")…),HVDCLine => Dict("DC1" => HVDCLine(name="DC1")),HydroDispatch => Dict("215_HYDRO_3" => HydroDispatch(name="215_HYDRO_3"),"215_HYDRO_2" => HydroDispatch(name="215_HYDRO_2"),"222_HYDRO_5" => HydroDispatch(name="222_HYDRO_5"),"122_HYDRO_2" => HydroDispatch(name="122_HYDRO_2"),"322_HYDRO_2" => HydroDispatch(name="322_HYDRO_2"),"222_HYDRO_2" => HydroDispatch(name="222_HYDRO_2"),"122_HYDRO_6" => HydroDispatch(name="122_HYDRO_6"),"322_HYDRO_3" => HydroDispatch(name="322_HYDRO_3"),"322_HYDRO_4" => HydroDispatch(name="322_HYDRO_4"),"222_HYDRO_6" => Hydro

In [12]:
print(summary(sys_DA))

System
Base Power: 100.0

Components
12×3 DataFrames.DataFrame
│ Row │ ConcreteType        │ SuperTypes                                                                              │ Count │
│     │ String              │ String                                                                                  │ Int64 │
├─────┼─────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ 1   │ Bus                 │ Topology <: Component <: PowerSystemType <: Any                                         │ 73    │
│ 2   │ GenericBattery      │ Storage <: Injection <: Device <: Component <: PowerSystemType <: Any                   │ 1     │
│ 3   │ HVDCLine            │ DCBranch <: Branch <: Device <: Component <: PowerSystemType <: Any                     │ 1     │
│ 4   │ HydroDispatch       │ HydroGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: Any     │ 20    │
│ 5   │ Line                │ ACBranch <:

In [13]:
 get_forecast_initial_times(sys_DA)

1-element Array{DateTime,1}:
 2020-01-01T00:00:00

In [14]:
function subset_forecasts!(sys, forecast_type, initial_time, from_time, to_time)
    fcs = get_forecasts(forecast_type, sys, initial_time)
    new_fcs = []
    for fc in fcs
        res = PSY.get_resolution(fc)
        new_fc = Deterministic(PSY.get_component(fc),
            PSY.get_label(fc), 
            res, 
            from_time,
            PSY.get_data(fc)[from_time:res:to_time])
        push!(new_fcs, new_fc)
    end
    [remove_forecast!(sys, fc) for fc in fcs]
    [PSY.add_forecast!(sys, new_fc) for new_fc in new_fcs]

end

subset_forecasts! (generic function with 1 method)

In [15]:
subset_forecasts!(sys_DA, Deterministic, 
    get_forecast_initial_times(sys_DA)[1], 
    DateTime("2020-04-04T00:00:00"), 
    DateTime("2020-04-12T23:00:00"));

┌ Info: Deleted forecast 1f36fde4-0505-44f3-9a8d-3b8d3d50ddcc
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast d5929c44-8eac-48c3-8056-fb7b8545127c
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast f675ebd2-aebb-4dee-aa22-303d3e08011a
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 98da5c35-7cb1-4204-b5c5-32a8bd3bcaf4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 4546d90b-1d0e-4dae-9db3-ec5f233e5700
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 92cfdcce-dcc7-4737-a5d4-38d2285d119e
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast af19936b-6577-42ad-9db4-e2158e8b50f5
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/

┌ Info: Deleted forecast 4b8056db-d45c-463d-9538-90e5c8c20f68
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast f5a416c4-9080-4753-abfc-9175dcd24dc1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 27fec6e1-e629-4648-8f24-08c94614d6de
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast cefa858b-9462-41b3-a888-348b6d8fdade
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast ea51746d-0323-46c4-98e7-e8908b0cf9a8
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 596f0402-f43b-4573-84da-59d9e23c4efd
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast ec2a318f-ffcb-4ac7-b14b-08c88113e197
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/

In [16]:
PSY.get_forecast_initial_times(sys_DA)

1-element Array{DateTime,1}:
 2020-04-04T00:00:00

In [17]:
split_forecasts!(sys_DA, 
    get_forecasts(Deterministic, sys_DA, PSY.get_forecast_initial_times(sys_DA)[1]),
    Dates.Hour(24),
    48)

┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-05T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-06T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-07T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00

┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="313_PV_2")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-09T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="313_PV_2")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="313_PV_2")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="313_PV_2")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00

┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="324_PV_3")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-05T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="324_PV_3")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-06T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="324_PV_3")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-07T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="324_PV_3")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00

┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="101_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-09T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="101_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="101_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="101_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00

┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="103_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-05T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="103_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-06T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="103_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-07T00:00:00
│   horizon = 48
│   component = RenewableDispatch(name="103_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00

...

┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00
│   horizon = 48
│   component = RenewableFix(name="313_RTPV_7")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-09T00:00:00
│   horizon = 48
│   component = RenewableFix(name="313_RTPV_7")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T00:00:00
│   horizon = 48
│   component = RenewableFix(name="313_RTPV_7")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:00:00
│   horizon = 48
│   component = RenewableFix(name="313_RTPV_7")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   horizon

┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   horizon = 48
│   component = RenewableFix(name="320_RTPV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-05T00:00:00
│   horizon = 48
│   component = RenewableFix(name="320_RTPV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-06T00:00:00
│   horizon = 48
│   component = RenewableFix(name="320_RTPV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-07T00:00:00
│   horizon = 48
│   component = RenewableFix(name="320_RTPV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00
│   horizon

┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00
│   horizon = 48
│   component = RenewableFix(name="320_RTPV_5")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-09T00:00:00
│   horizon = 48
│   component = RenewableFix(name="320_RTPV_5")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T00:00:00
│   horizon = 48
│   component = RenewableFix(name="320_RTPV_5")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:00:00
│   horizon = 48
│   component = RenewableFix(name="320_RTPV_5")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   horizon

┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   horizon = 48
│   component = RenewableFix(name="118_RTPV_6")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-05T00:00:00
│   horizon = 48
│   component = RenewableFix(name="118_RTPV_6")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-06T00:00:00
│   horizon = 48
│   component = RenewableFix(name="118_RTPV_6")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-07T00:00:00
│   horizon = 48
│   component = RenewableFix(name="118_RTPV_6")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00
│   horizon

┌ Info: Created forecast with
│   initial_time = 2020-04-08T00:00:00
│   horizon = 48
│   component = RenewableFix(name="118_RTPV_10")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-09T00:00:00
│   horizon = 48
│   component = RenewableFix(name="118_RTPV_10")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T00:00:00
│   horizon = 48
│   component = RenewableFix(name="118_RTPV_10")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:00:00
│   horizon = 48
│   component = RenewableFix(name="118_RTPV_10")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   hor

In [18]:
print(summary(sys_DA))

System
Base Power: 100.0

Components
12×3 DataFrames.DataFrame
│ Row │ ConcreteType        │ SuperTypes                                                                              │ Count │
│     │ String              │ String                                                                                  │ Int64 │
├─────┼─────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ 1   │ Bus                 │ Topology <: Component <: PowerSystemType <: Any                                         │ 73    │
│ 2   │ GenericBattery      │ Storage <: Injection <: Device <: Component <: PowerSystemType <: Any                   │ 1     │
│ 3   │ HVDCLine            │ DCBranch <: Branch <: Device <: Component <: PowerSystemType <: Any                     │ 1     │
│ 4   │ HydroDispatch       │ HydroGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: Any     │ 20    │
│ 5   │ Line                │ ACBranch <:

In [19]:
subset_forecasts!(sys_RT, Deterministic, 
    get_forecast_initial_times(sys_RT)[1], 
    DateTime("2020-04-04T00:00:00"), 
    DateTime("2020-04-12T23:00:00"));

┌ Info: Deleted forecast a3ed9c34-c329-4eee-a5e5-0501a76f6c48
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast f8f5b1df-a814-4d8b-bbb7-fcedeabfed57
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 2fc25688-a249-4b33-ad86-96586a2d67db
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 9dd8d01c-cebe-4d2f-a32a-d8dd1ba99e0e
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast b3b97503-d090-4deb-b653-ab0e10e9c097
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast a19fe019-24c9-4e31-8ab9-285ce1187ed2
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast d9377890-5dc3-4fea-a21e-b570f87770ed
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/

┌ Info: Deleted forecast 711ecfdc-7478-499a-bb30-3e564a2d26c9
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 1511fc7f-1f41-4c7d-bd46-20738651ff96
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast f16d5ba8-d4e6-4f4f-8086-66ed6e6d4060
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast db0f2fae-9aaa-419d-81d9-255bcf0cc52b
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 1cfde3cd-f866-4372-86e2-b02780e8fbce
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast 4ea30686-e722-412f-906a-51a5e317a9e1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/base.jl:651
┌ Info: Deleted forecast a3d00866-d5d0-4169-aea5-b05acb28d574
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/

In [20]:
split_forecasts!(sys_RT, 
    get_forecasts(Deterministic, sys_RT, get_forecast_initial_times(sys_RT)[1]),
    Dates.Minute(5),
    12)

┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:00:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:05:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:10:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:15:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T00:20:00

┌ Info: Created forecast with
│   initial_time = 2020-04-04T03:40:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T03:45:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T03:50:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T03:55:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T04:00:00

┌ Info: Created forecast with
│   initial_time = 2020-04-04T07:20:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T07:25:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T07:30:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T07:35:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T07:40:00

┌ Info: Created forecast with
│   initial_time = 2020-04-04T11:00:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T11:05:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T11:10:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T11:15:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T11:20:00

┌ Info: Created forecast with
│   initial_time = 2020-04-04T14:40:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T14:45:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T14:50:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T14:55:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-04T15:00:00

...

┌ Info: Created forecast with
│   initial_time = 2020-04-10T21:00:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T21:05:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T21:10:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T21:15:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-10T21:20:00

┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:40:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:45:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:50:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T00:55:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T01:00:00

┌ Info: Created forecast with
│   initial_time = 2020-04-11T04:20:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T04:25:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T04:30:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T04:35:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T04:40:00

┌ Info: Created forecast with
│   initial_time = 2020-04-11T08:00:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T08:05:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T08:10:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T08:15:00
│   horizon = 12
│   component = RenewableDispatch(name="320_PV_1")
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/7bCPK/src/models/forecasts.jl:381
┌ Info: Created forecast with
│   initial_time = 2020-04-11T08:20:00

Excessive output truncated after 524394 bytes.

In [20]:
# TODO: load from here.
to_json(sys_DA,"sys_DA_44_412.json")
to_json(sys_RT,"sys_RT_44_412.json")

## Setup simulations

In [21]:
## UC Model Ref
branches = Dict{Symbol, DeviceModel}(#:L => DeviceModel(PSY.Line, PSI.StaticLine),
                                     #:T => DeviceModel(PSY.Transformer2W, PSI.StaticTransformer),
                                     #:TT => DeviceModel(PSY.TapTransformer, PSI.StaticTransformer),
                                     #:dc_line => DeviceModel(PSY.HVDCLine, PSI.HVDCDispatch)
                                    )

services = Dict{Symbol, PSI.ServiceModel}()

devices = Dict{Symbol, DeviceModel}(:Generators => DeviceModel(PSY.ThermalStandard, PSI.ThermalBasicUnitCommitment),
                                    :Ren => DeviceModel(PSY.RenewableDispatch, PSI.RenewableFullDispatch),
                                    :Loads =>  DeviceModel(PSY.PowerLoad, PSI.StaticPowerLoad),
                                    #:ILoads =>  DeviceModel(PSY.InterruptibleLoad, PSI.StaticPowerLoad),
                                    )       


model_ref_uc= ModelReference(CopperPlatePowerModel, devices, branches, services);

In [22]:
## ED Model Ref
branches = Dict{Symbol, DeviceModel}(#:L => DeviceModel(PSY.Line, PSI.StaticLine),
                                     #:T => DeviceModel(PSY.Transformer2W, PSI.StaticTransformer),
                                     #:TT => DeviceModel(PSY.TapTransformer, PSI.StaticTransformer),
                                     #:dc_line => DeviceModel(PSY.HVDCLine, PSI.HVDCDispatch)
                                        )

services = Dict{Symbol, PSI.ServiceModel}()

devices = Dict{Symbol, DeviceModel}(:Generators => DeviceModel(PSY.ThermalStandard, PSI.ThermalDispatch, SemiContinuousFF(:P, :ON)),
                                    :Ren => DeviceModel(PSY.RenewableDispatch, PSI.RenewableFullDispatch),
                                    :Loads =>  DeviceModel(PSY.PowerLoad, PSI.StaticPowerLoad),
                                    #:ILoads =>  DeviceModel(PSY.InterruptibleLoad, PSI.DispatchablePowerLoad),
                                    )       

model_ref_ed= ModelReference(CopperPlatePowerModel, devices, branches, services);

In [23]:
stages = Dict(1 => Stage(model_ref_uc, 1, sys_DA, GLPK_optimizer),
              2 => Stage(model_ref_ed, 24, sys_RT, GLPK_optimizer,  Dict(1 => Synchronize)))

Dict{Int64,Stage} with 2 entries:
  2 => Simulation Stage…
  1 => Simulation Stage…

In [24]:
sim = Simulation("test", 1, stages, "/Users/cbarrows/Downloads/"; verbose = true, system_to_file = false)

┌ Info: Building Stage 2
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/UMU6S/src/core/build_simulations.jl:81
┌ Info: Building ThermalStandard with ThermalDispatch formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/UMU6S/src/core/build_operations.jl:14
┌ Warning: The cost function provided for P_{322_CT_6, 1} device is not compatible with a linear PWL cost function.
│         An SOS-2 formulation will be added to the model.
│         This will result in additional binary variables added to the model.
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/UMU6S/src/devices/device_models/common/cost_functions.jl:268


ErrorException: Constraints of type MathOptInterface.VectorOfVariables-in-MathOptInterface.SOS2{Float64} are not supported by the solver and there are no bridges that can reformulate it into supported constraints.

In [28]:
res = run_sim_model!(sim)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/cbarrows/.julia/packages/MathOptInterface/016Ol/src/Utilities/copy.jl:140
┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/cbarrows/.julia/packages/MathOptInterface/016Ol/src/Utilities/copy.jl:140


ArgumentError: ArgumentError: `nothing` should not be printed; use `show`, `repr`, or custom output instead.